# 혼공머 07-2
> 심층 신경망
- toc:true
- branch: master
- badges: true
- comments: true
- author: 김동준
- categories : ["Python", "혼공머"]

# 주로쓰는 패키지들

In [1]:
#collapse
import numpy as np #넘파이
import pandas as pd #판다스
from plotnine import *  #플롯나인
import matplotlib.pyplot as plt #맷플랏립
import plotly.express as px #플랏리 상호작용 그래프
from IPython.display import HTML #블로그에 html로 올리려고 변환하는 패키지
import seaborn as sns # 씨본, 히스토그램 깔끔하게 그리는 패키지
#___________________________________

from sklearn.neighbors import KNeighborsClassifier # k 최근접이웃
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor  # 결정계수 
from sklearn.metrics import mean_absolute_error # 타깃과 예측의 절댓값 오차 평균을 반환
from sklearn.linear_model import LinearRegression # 선형 회귀
from sklearn.preprocessing import PolynomialFeatures #다중회귀로의 변환기
from sklearn.preprocessing import StandardScaler #규제
from sklearn.linear_model import Ridge #릿지
from sklearn.linear_model import Lasso #라쏘
from sklearn.linear_model import LogisticRegression # 로지스틱회귀
from scipy.special import expit #시그모이드함수
from scipy.special import softmax #소프트맥스함수
from sklearn.linear_model import SGDClassifier # 확률적 경사 하강법
from sklearn.tree import DecisionTreeClassifier # 트리
from sklearn.tree import plot_tree # 트리 모형
from sklearn.model_selection import cross_validate # 교차 검증
from sklearn.model_selection import StratifiedKFold # Kfold 교차 검증
from sklearn.model_selection import GridSearchCV # 그리드 서치 (하이퍼 파라미터 튜닝)
from scipy.stats import uniform, randint #랜덤 서치
from sklearn.model_selection import RandomizedSearchCV # 랜덤 서치 클래스
from sklearn.ensemble import RandomForestClassifier # 랜덤포레스트 앙상블
from sklearn.ensemble import ExtraTreesClassifier # 엑스트라 트리 앙상블
from sklearn.ensemble import GradientBoostingClassifier # 그레이디언트 부스팅 앙상블

# 히스토그램 기반 그레이디언트 부스팅
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.inspection import permutation_importance # 특성중요도
from xgboost import XGBClassifier # 알고리즘을 구현한 또다른 라이브러리1
from lightgbm import LGBMClassifier # 알고리즘을 구현한 또다른 라이브러리2 , 마이크로소프트에서 구현 


# ____________________
from sklearn.cluster import KMeans # KMeans
from sklearn.decomposition import PCA # 주성분 분석

# 7장 딥러닝
from tensorflow import keras # 케라스 
import tensorflow as tf # 텐서플로


# 2개의 층

In [2]:
# 케라스 API 로 패션 MNIST 데이터셋 로드
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

In [3]:
# 1. 이미지의 픽셀값 0~255 범위에서 0~1로 전처리
# 2. 28x28 크기의 2차원 배열을 784 크기의 1차원 배열로 펼치기
# 3. 훈련세트와 검증세트 나누기

train_scaled = train_input / 255.0
train_scaled = train_scaled.reshape(-1, 28*28)
train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42)

## 만든 인공 신경망 모형에 2개의 층 추가

> 입력층과 출력층 사이에 밀집층이 추가된 것. $\to$ 은닉층 이라고 부름.

> 은닉층에는 활성화 함수가 있음 (신경망 층의 선형 방정식의 계산 값에 적용하는 함수)

> 소프트 맥스 함수도 활성화 함수

> 출력층에 적용하는 활성화 함수는 종류가 제한 (이진 분류 : 시그모이드 함수, 다중 분류 : 소프트맥스 함수)

> 분류 문제는 클래스에 대한 확률을 출력하기 위해 활성화 함수를 사용, 회귀의 출력은 임의의 어떤 숫자라서 활성화 함수를 적용할 필요가 없음. \
즉, 출력층의 선형 방정식의 계산을 그대로 출력

> 은닉층에서 선형적인 산술 계산만 수행한다면 수행 역할이 없는 셈이라서 활성화 함수 사용

In [5]:
# 시그모이드 활성화 함수를 사용한 은닉층과 소프트맥스 함수를 사용한 출력층 만들기
dense1 = keras.layers.Dense(100, activation='sigmoid', input_shape=(784,))
dense2 = keras.layers.Dense(10, activation='softmax')

> dense1 이 은닉층이고 100개의 뉴런을 가진 밀집층, sigmoid로 활성화 함수 지정, 출력층의 뉴런보다는 많게 만들어야함.

> dense2 : 출력층

# 심층 신경망 만들기 (Deep Neural network)

In [6]:
# 심층 신경망 생성
model = keras.Sequential([dense1, dense2])

> 리스트로 전달 해야하며 , 출력층을 가장 마지막에 두어야 한다.

In [7]:
# 서머리로 유용한 정보 확인 (케라스)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               78500     
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


> 모델의 이름이 나오며 , 그 모델에 들어 있는 층이 순서대로 나열 (은닉층에서 출력층의 순서)

> 층마다 층 이름, 클래스 ,출력 크기 ,모델 파라미터 개수가 출력

> Output Shape : (샘플의 수 , 뉴런의 수)

> Total params : 총 모델 파라미터 개수

> Trainable params : 훈련되는 파라미터 개수

# 층을 추가하는 다른 방법

In [8]:
# Seqeuntial 클래스에 층을 추가하는 다른 방법.(Seqeuntial 클래스의 생성자 안에서 바로 Dense 클래스의 객체를 생성)

model = keras.Sequential([
    keras.layers.Dense(100, activation='sigmoid', input_shape=(784,), name='hidden'),
    keras.layers.Dense(10, activation='softmax', name='output')
], name='패션 MNIST 모델')

> 추가되는 층을 한눈에 쉽게 알아볼 수 있는 장점이 있음.

> 층의 이름은 반드시 영문이어야 함.

In [9]:
model.summary()

Model: "패션 MNIST 모델"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden (Dense)              (None, 100)               78500     
                                                                 
 output (Dense)              (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


> 모델 이름과 층이름이 바뀜

In [10]:
# 모델의 add()메서드로 층 추가하기
model = keras.Sequential()
model.add(keras.layers.Dense(100, activation='sigmoid', input_shape=(784,)))
model.add(keras.layers.Dense(10, activation='softmax'))

In [11]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 100)               78500     
                                                                 
 dense_3 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [12]:
# 모델 훈련 ( compile() 메서드 )
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')

model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.5661 - accuracy: 0.8092
Epoch 2/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.4068 - accuracy: 0.8532
Epoch 3/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3728 - accuracy: 0.8646
Epoch 4/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3515 - accuracy: 0.8725
Epoch 5/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3335 - accuracy: 0.8782


> 추가된 층이 성능을 향상시켰다는 것을 알 수 있다.

> 몇개의 층이 추가되어도 compile() 과 fit() 메서드의 사용법은 동일하다.

# 렐루 함수  (활성화 함수)

> 시그모이드 함수는 층이 많을수록 효과가 누적되어 학습을 더 어렵게 만듬\
$\to$ 해결 : 렐루 함수\
$\to$ 입력이 양수일 경우 마치 활성화 함수가 없는 것처럼 그냥 입력을 통과 시키고 음수일 경우에는 0으로 만듬

In [13]:
# Flatten 층을 입력층 바로 뒤에 추가
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

> 첫 번째 Dense 층에 있던 input_shape 매개변수를 Flatten 층으로 옮김

> 첫 번째 Dense 층의 활성화 함수를 relu로 바꿈

> Flatten 클래스는 학습하는 층이 아님

In [14]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense_4 (Dense)             (None, 100)               78500     
                                                                 
 dense_5 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


> flatten 클래스에 포함된 모델 파라미터는 0개

> 케라스dml flatten 층을 신경망 모형에 추가하면 입력값의 차원을 짐작할 수 있다.

> 784개의 입력이 첫 번째 은닉층에 전달되었다.

> 입력 데이터에 대한 전처리 과정을 가능한 모델에 포함 시킨다. : 케라스 API

In [15]:
# 다시 모델 훈련
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

train_scaled = train_input / 255.0

train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42)

In [16]:
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')

model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.5315 - accuracy: 0.8127
Epoch 2/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3930 - accuracy: 0.8584
Epoch 3/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3572 - accuracy: 0.8723
Epoch 4/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3345 - accuracy: 0.8813
Epoch 5/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3215 - accuracy: 0.8850


> 시그모이드 함수를 사용했을 때와 비교하면 성능이 조금 향상

In [17]:
# 검증 세트에서의 성능도 확인
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 0s 897us/step - loss: 0.3641 - accuracy: 0.8789


[0.36406102776527405, 0.8789166808128357]

> 은닉층을 추가하지 않은 경우보다 몇 퍼센트 성능이 향상

# 옵티마이저

> 은닉층의 뉴런 개수도 하이퍼 파라미터이다.

> 층의 종류도 하이퍼 파라미터 이다.

> 옵티마이저 : 케라스에서 제공하는 다양한 경사 하강법 알고리즘

> RMSprop의 학습률 또한 하이터 파라미터

## 가장 기본적인 옵티마이저 : 확률적 경사 하강법 SGD

In [18]:
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics='accuracy')

In [19]:
sgd = keras.optimizers.SGD()
model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics='accuracy')

In [20]:
# 학습률 변경
sgd = keras.optimizers.SGD(learning_rate=0.1)

## 모멘텀 최적화

> 그레이디언트를 가속도처럼 사용

> SGD 클래스의 nesterov 매개변수를 기본값 False에서 True로 바꾸면 네스테로프 모멘텀 최적화 를 사용

In [21]:
sgd = keras.optimizers.SGD(momentum=0.9, nesterov=True)

## 적응적 학습률

In [22]:
adagrad = keras.optimizers.Adagrad()
model.compile(optimizer=adagrad, loss='sparse_categorical_crossentropy', metrics='accuracy')

In [23]:
rmsprop = keras.optimizers.RMSprop()
model.compile(optimizer=rmsprop, loss='sparse_categorical_crossentropy', metrics='accuracy')

In [24]:
# Adam 클래스의 매개변수 기본값을 사용해 패션  MNIST 모델훈련
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [25]:
# compile() 메서드의 optimizer를 adam 으로 설정하고 5번의 에포크 동안 훈련
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')

model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.5299 - accuracy: 0.8139
Epoch 2/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3942 - accuracy: 0.8590
Epoch 3/5
1500/1500 [==============================] - 1s 974us/step - loss: 0.3546 - accuracy: 0.8698
Epoch 4/5
1500/1500 [==============================] - 1s 990us/step - loss: 0.3245 - accuracy: 0.8810
Epoch 5/5
1500/1500 [==============================] - 1s 984us/step - loss: 0.3063 - accuracy: 0.8852


In [26]:
# 검증 세트 성능 확인
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 0s 729us/step - loss: 0.3352 - accuracy: 0.8794


[0.3352320194244385, 0.8794166445732117]

_________

# **마무리**

`*` 키워드로 끝내는 핵심 포인트

`-` **심층 신경망** : 2개 이상의 층을 포함한 신경망입니다. 종종 다층 인공 신경망, 심층 신경망, 딥러닝을 같은 의미로 사용합니다.

`-` **렐루 함수** : 이미지 분류 모델의 은닉층에 많이 사용하는 활성화 함수입니다. 시그모이드 함수는 층이 많을수록 활성화 함수의 양쪽 끝에서 변화가 작기 때문에 학습이 어려워집니다. 렐루함수는 이런 문제가 없으며 계산도 간단합니다.


`-` **옵티마이저** : 신경망의 가중치와 절편을 학습하기 위한 알고리즘 또는 방법을 말합니다. 케라스에는 다양한 경사 하강법 알고리즘이 구현되어 있습니다. 대표적으로 SGD, 네스테로프 모멘텀, RMSprop, Adam 등이 있습니다.

`-` **핵심 패키지**

`-` TensorFlow

> `1` add() : 케라스 모델에 층을 추가하는 메서드입니다.

>>케라스 모델의 add() 메서드는 keras.layers 패키지 아래에 있는 층의 객체를 입력받아 신경망 모델에 추가합니다. add() 메서드를 호출하여 전달한 순서대로 층이 차례대로 늘어납니다.


> `2` summary () : 케라스 모델의 정보를 출력하는 메서드입니다.

>>모델에 추가된 층의 종류와 순서, 모델 파라미터 개수를 출력합니다. 층을 만들 때 name 매개변수로 이름을 지정하면 summary() 메서드 출력에서 구분하기 쉽습니다.

>>percentiles 매개변수에서 백분위수를 지정합니다. 기본값은 [0.25, 0.5, 0.75]입니다.


> `3` SGD : 기본 경사 하강법 옵티마이저 클래스입니다.

>>learning_rate 매개변수로 학습률을 지정하며 기본값은 0.01입니다

>>momentum 매개변수에 이 이상의 값을 지정하면 모멘텀 최적화를 수행합니다.

>>nesteroy 매개변수를 True로 설정하면 네스테로프 모멘텀 최적화를 수행합니다.


> `4` Adagrad : Adagrad 옵티마이저 클래스입니다.

>>learning_rate 매개변수로 학습률을 지정하며 기본값은 0.001입니다. learning_rate 매개변수로 학습률을 지정하며 기본값은 0.001입니다. learning_rate 매개변수로 학습률을 지정하며 기본값은 0.001입니다.

>>Adagrad는 그레이디언트 제곱을 누적하여 학습률을 나눕니다. initial accumulator Value 매개변수에서 누적 초깃값을 지정할 수 있으며 기본값은 0.1입니다.


> `5` RMSprop : RMSprop 옵티마이저 클래스입니다.

>>Adagrad처럼 그레이디언트 제곱으로 학습률을 나누지만 최근의 그레이디언트를 사용하기 위해 지수 감소를 사용합니다. rho 매개변수에서 감소 비율을 지정하며 기본값은 0.9입니다.


> `6` Adam : Adam 옵티마이저 클래스입니다.

>>모멘텀 최적화에 있는 그레이디언트의 지수 감소 평균을 조절하기 위해 beta 1 매개변수가 있으며 기본값은 0.9 입니다.

>>RMSprop에 있는 그레이디언트 제곱의 지수 감소 평균을 조절하기 위해 beta 2 매개변수가 있으며 기본값은 0.999입니다.